In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-03-05 06:29:20.555475: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:

import cv2
import os
import numpy as np

In [9]:


def loadImages(path, urls,target ):
  images = []
  labels = []
  #for i in range(len(urls))
  for i in range(len(urls)):
    img_path = path + "/" + urls[i]
    img = cv2.imread(img_path)
    img = img / 255.0
    # if we want to resize the images
    img = cv2.resize(img, (100, 100))
    images.append(img)
    labels.append(target)
  images = np.asarray(images)
  return images, labels


covid_path = "./../data/COVID-19_Radiography_Dataset/COVID/images"
covidUrl = os.listdir(covid_path)
covidImages, covidTargets = loadImages(covid_path, covidUrl, 1)

normal_path = "./../data/COVID-19_Radiography_Dataset/Normal/images"
normal_urls = os.listdir(normal_path)
normalImages, normalTargets = loadImages(normal_path, normal_urls, 0)

covidImages=np.asarray(covidImages)

normalImages=np.asarray(normalImages)

data = np.r_[covidImages, normalImages]

targets = np.r_[covidTargets, normalTargets]


from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(data, targets, test_size=0.25)


In [19]:
# Load a pre-trained model
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(100, 100, 3))

# Freeze all layers except the last one
for layer in base_model.layers:
    layer.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(1024, activation="relu")(x)
predictions = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 106, 106, 3)  0           ['input_5[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 50, 50, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 50, 50, 64)   256         ['conv1_conv[0][0]']       

In [20]:
# train the model
model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test))

Epoch 1/15
324/324 [==============================] - 11s 23ms/step - loss: 0.5863 - accuracy: 0.7218 - val_loss: 0.5030 - val_accuracy: 0.7500
Epoch 2/15
324/324 [==============================] - 5s 17ms/step - loss: 0.5119 - accuracy: 0.7461 - val_loss: 0.4856 - val_accuracy: 0.7575
Epoch 3/15
324/324 [==============================] - 5s 17ms/step - loss: 0.4900 - accuracy: 0.7536 - val_loss: 0.5006 - val_accuracy: 0.7373
Epoch 4/15
324/324 [==============================] - 5s 17ms/step - loss: 0.4734 - accuracy: 0.7614 - val_loss: 0.5378 - val_accuracy: 0.7265
Epoch 5/15
324/324 [==============================] - 6s 17ms/step - loss: 0.4647 - accuracy: 0.7628 - val_loss: 0.4290 - val_accuracy: 0.7819
Epoch 6/15
324/324 [==============================] - 5s 17ms/step - loss: 0.4658 - accuracy: 0.7639 - val_loss: 0.4373 - val_accuracy: 0.7764
Epoch 7/15
324/324 [==============================] - 6s 17ms/step - loss: 0.4413 - accuracy: 0.7769 - val_loss: 0.4443 - val_accuracy: 0.794

In [21]:
# evaluate the model
_, train_acc = model.evaluate(x_train, y_train, verbose=0)
_, test_acc = model.evaluate(x_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.834, Test: 0.834
